In [52]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline



Quantum Generative Adversarial Networks with Cirq + TensorFlow
==============================================================

.. meta::
    :property="og:description": This demo constructs and trains a Quantum
        Generative Adversarial Network (QGAN) using PennyLane, Cirq, and TensorFlow.
    :property="og:image": https://pennylane.ai/qml/_images/qgan3.png

This demo constructs a Quantum Generative Adversarial Network (QGAN)
(`Lloyd and Weedbrook
(2018) <https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.121.040502>`__,
`Dallaire-Demers and Killoran
(2018) <https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.012324>`__)
using two subcircuits, a *generator* and a *discriminator*. The
generator attempts to generate synthetic quantum data to match a pattern
of "real" data, while the discriminator tries to discern real data from
fake data (see image below). The gradient of the discriminator’s output provides a
training signal for the generator to improve its fake generated data.

|

.. figure:: ../demonstrations/QGAN/qgan.png
    :align: center
    :width: 75%
    :target: javascript:void(0)

|


Using Cirq + TensorFlow
~~~~~~~~~~~~~~~~~~~~~~~
PennyLane allows us to mix and match quantum devices and classical machine
learning software. For this demo, we will link together
Google's `Cirq <https://cirq.readthedocs.io/en/stable/>`_ and `TensorFlow <https://www.tensorflow.org/>`_ libraries.

We begin by importing PennyLane, NumPy, and TensorFlow.



In [53]:
import pennylane as qml
import numpy as np
import tensorflow as tf

We also declare a 3-qubit simulator device running in Cirq.



In [54]:
dev = qml.device('cirq.simulator', wires=4)

Generator and Discriminator
~~~~~~~~~~~~~~~~~~~~~~~~~~~

In classical GANs, the starting point is to draw samples either from
some “real data” distribution, or from the generator, and feed them to
the discriminator. In this QGAN example, we will use a quantum circuit
to generate the real data.

For this simple example, our real data will be a qubit that has been
rotated (from the starting state $\left|0\right\rangle$) to some
arbitrary, but fixed, state.



In [117]:
def real(angles, **kwargs):
    qml.RY(0.27740551, wires=0)
    qml.PauliX(wires =0)
    qml.CRY(0.20273270, wires=[0, 1])
    qml.PauliX(wires =0)
    qml.CRY(np.pi/2, wires=[0, 1])
    qml.CRY(np.pi/2, wires=[0, 2])
    qml.PauliX(wires =0)
    qml.CRY(1.42492, wires=[0, 2])
    qml.PauliX(wires =0)

For the generator and discriminator, we will choose the same basic
circuit structure, but acting on different wires.

Both the real data circuit and the generator will output on wire 0,
which will be connected as an input to the discriminator. Wire 1 is
provided as a workspace for the generator, while the discriminator’s
output will be on wire 2.



In [118]:
def generator(w, **kwargs):
    qml.Hadamard(wires=0)
#     qml.Hadamard(wires=1)
    
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RX(w[2], wires=2)
    
    qml.RY(w[3], wires=0)
    qml.RY(w[4], wires=1)
    qml.RY(w[5], wires=2)
    
    qml.RZ(w[6], wires=0)
    qml.RZ(w[7], wires=1)
    qml.RZ(w[8], wires=2)
    
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    
    qml.RX(w[9], wires=0)
    qml.RY(w[10], wires=0)
    qml.RZ(w[11], wires=0)
    
    qml.RX(w[12], wires=1)
    qml.RY(w[13], wires=1)
    qml.RZ(w[14], wires=1)


def discriminator(w):
    qml.Hadamard(wires=1)

    qml.RX(w[0], wires=1)
    qml.RX(w[1], wires=2)
    qml.RX(w[2], wires=3)
    
    qml.RY(w[3], wires=1)
    qml.RY(w[4], wires=2)
    qml.RY(w[5], wires=3)
    
    qml.RZ(w[6], wires=1)
    qml.RZ(w[7], wires=2)
    qml.RZ(w[8], wires=3)
    
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    
    qml.RX(w[9], wires=2)
    qml.RY(w[10], wires=2)
    qml.RZ(w[11], wires=2)
    
    qml.RX(w[12], wires=3)
    qml.RY(w[13], wires=3)
    qml.RZ(w[14], wires=3)

We create two QNodes. One where the real data source is wired up to the
discriminator, and one where the generator is connected to the
discriminator. In order to pass TensorFlow Variables into the quantum
circuits, we specify the ``"tf"`` interface.



In [119]:
@qml.qnode(dev, interface="tf")
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi, theta, omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(3))


@qml.qnode(dev, interface="tf")
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(3))

QGAN cost functions
~~~~~~~~~~~~~~~~~~~

There are two cost functions of interest, corresponding to the two
stages of QGAN training. These cost functions are built from two pieces:
the first piece is the probability that the discriminator correctly
classifies real data as real. The second piece is the probability that the
discriminator classifies fake data (i.e., a state prepared by the
generator) as real.

The discriminator is trained to maximize the probability of
correctly classifying real data, while minimizing the probability of
mistakenly classifying fake data.

The generator is trained to maximize the probability that the
discriminator accepts fake data as real.



In [120]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true


def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

Training the QGAN
~~~~~~~~~~~~~~~~~

We initialize the fixed angles of the “real data” circuit, as well as
the initial parameters for both generator and discriminator. These are
chosen so that the generator initially prepares a state on wire 0 that
is very close to the $\left| 1 \right\rangle$ state.



In [121]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 14) + \
                   np.random.normal(scale=eps, size=(15,))
init_disc_weights = np.random.normal(size=(15,))

gen_weights = tf.Variable(init_gen_weights)
disc_weights = tf.Variable(init_disc_weights)
print(gen_weights)

<tf.Variable 'Variable:0' shape=(15,) dtype=float64, numpy=
array([ 3.15923318e+00,  4.00157208e-03,  9.78737984e-03,  2.24089320e-02,
        1.86755799e-02, -9.77277880e-03,  9.50088418e-03, -1.51357208e-03,
       -1.03218852e-03,  4.10598502e-03,  1.44043571e-03,  1.45427351e-02,
        7.61037725e-03,  1.21675016e-03,  4.43863233e-03])>


We begin by creating the optimizer:



In [122]:
opt = tf.keras.optimizers.SGD(0.4)

In the first stage of training, we optimize the discriminator while
keeping the generator parameters fixed.



In [123]:
cost = lambda: disc_cost(disc_weights)

for step in range(100):
    opt.minimize(cost, disc_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.0019399821758270264
Step 5: cost = -0.0076020509004592896
Step 10: cost = -0.027170330286026
Step 15: cost = -0.1009305864572525
Step 20: cost = -0.31306858360767365
Step 25: cost = -0.5063612759113312
Step 30: cost = -0.576730165630579
Step 35: cost = -0.6141441911458969
Step 40: cost = -0.633061483502388
Step 45: cost = -0.6403361186385155
Step 50: cost = -0.6427197977900505
Step 55: cost = -0.643457680940628
Step 60: cost = -0.6436858028173447
Step 65: cost = -0.6437596529722214
Step 70: cost = -0.6437873989343643
Step 75: cost = -0.6438009217381477
Step 80: cost = -0.6438104659318924
Step 85: cost = -0.6438182517886162
Step 90: cost = -0.6438257843255997
Step 95: cost = -0.6438328251242638


At the discriminator’s optimum, the probability for the discriminator to
correctly classify the real data should be close to one.



In [124]:
print("Prob(real classified as real): ", prob_real_true(disc_weights).numpy())

Prob(real classified as real):  0.8142546266317368


For comparison, we check how the discriminator classifies the
generator’s (still unoptimized) fake data:



In [125]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  0.17041606456041336


In the adversarial game we now have to train the generator to better
fool the discriminator. For this demo, we only perform one stage of the
game. For more complex models, we would continue training the models in an
alternating fashion until we reach the optimum point of the two-player
adversarial game.



In [126]:
cost = lambda: gen_cost(gen_weights)

for step in range(100):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.17156939208507538
Step 5: cost = -0.19378571957349777
Step 10: cost = -0.3354378044605255
Step 15: cost = -0.7252762317657471
Step 20: cost = -0.9431019332259893
Step 25: cost = -0.9877590350806713
Step 30: cost = -0.996944987680763
Step 35: cost = -0.9991390050272457
Step 40: cost = -0.9997334669315023
Step 45: cost = -0.9999113798548933
Step 50: cost = -0.9999688424322812
Step 55: cost = -0.999988498377661
Step 60: cost = -0.9999957134946271
Step 65: cost = -0.9999981789099479
Step 70: cost = -0.9999995012087197
Step 75: cost = -0.9999998565759824
Step 80: cost = -1.0000001788959167
Step 85: cost = -1.0000002135157224
Step 90: cost = -1.0000001683291586
Step 95: cost = -1.000000233990138


At the optimum of the generator, the probability for the discriminator
to be fooled should be close to 1.



In [127]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  1.000000295793245


At the joint optimum the discriminator cost will be close to zero,
indicating that the discriminator assigns equal probability to both real and
generated data.



In [128]:
print("Discriminator cost: ", disc_cost(disc_weights).numpy())

Discriminator cost:  0.18574566916150825


The generator has successfully learned how to simulate the real data
enough to fool the discriminator.

Let's conclude by comparing the states of the real data circuit and the generator. We expect
the generator to have learned to be in a state that is very close to the one prepared in the
real data circuit. An easy way to access the state of the first qubit is through its
`Bloch sphere <https://en.wikipedia.org/wiki/Bloch_sphere>`__ representation:



In [129]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Real Bloch vector: [0.21168673 0.         0.96176927]
Generator Bloch vector: [ 0.989946    0.02437505 -0.13933229]


In [130]:
print(gen_weights)

<tf.Variable 'Variable:0' shape=(15,) dtype=float64, numpy=
array([ 3.15923318e+00, -1.07908023e-03,  1.87523149e-02,  1.38386948e-01,
        1.57086483e+00, -1.38184036e-04,  9.50113371e-03, -1.07914958e-03,
       -3.13009652e-03,  4.10598502e-03,  1.44043571e-03,  1.45427351e-02,
       -5.30614464e-02, -2.20538651e+00, -2.64677566e-02])>


In [131]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
w = gen_weights.numpy()
q = QuantumRegister(3)
c = ClassicalRegister(3)
qc = QuantumCircuit(q, c)
qc.h(q[0])
qc.rx(w[0], q[0])
qc.rx(w[1], q[1])
qc.rx(w[2], q[2])

qc.ry(w[3], q[0])
qc.ry(w[4], q[1])
qc.ry(w[5], q[2])

qc.rz(w[6], q[0])
qc.rz(w[7], q[1])
qc.rz(w[8], q[2])

qc.cx(q[0], q[1])
qc.cx(q[1], q[2])

qc.rx(w[9], q[0])
qc.ry(w[10], q[0])
qc.rz(w[11], q[0])

qc.rx(w[12], q[1])
qc.ry(w[13], q[1])
qc.rz(w[14], q[1])

In [132]:
job = execute(qc, backend=Aer.get_backend("statevector_simulator"))
vec = job.result().get_statevector()
for i in vec:
    print(i)

(-0.20678143981403752+0.03258496547420384j)
(-0.2387806803175792+0.031624672178151486j)
(0.403455573841265-0.09249549020817205j)
(0.4667087529950727-0.09496931628045874j)
(-0.40710720182628585+0.07473240481435375j)
(-0.4703416105478664+0.07442328401141919j)
(-0.20288683573444108+0.051936559203879934j)
(-0.2348167048211411+0.05398754465103862j)
